Note book to assess precipitation efficiency in Rosi's MPAS aquaplanet simulations

### Main settings

In [5]:
from matplotlib import pyplot as plt
from matplotlib import colors
import seaborn as sns
import pickle
from scipy import stats
import numpy as np

#### Read data

In [2]:
# Get list of files
def get_file_list():
    import subprocess
    data_path = '/glade/campaign/mmm/dpm/rberrios/glade_scratch/MPAS_APE/aqua_sstmax10N_ASD/CTL/TC_3km/'
    process = subprocess.Popen(['ls '+data_path+'waterPaths.2000-05-0*'],shell=True,
        stdout=subprocess.PIPE,universal_newlines=True)
    file_list = process.stdout.readlines()
    for ifile in range(len(file_list)):
        file_list[ifile] = file_list[ifile].strip()
        # Split to retain only the file name, remove path
        file_list[ifile] = file_list[ifile].split('/')[-1]
    process = subprocess.Popen(['ls '+data_path+'waterPaths.2000-05-10*'],shell=True,
        stdout=subprocess.PIPE,universal_newlines=True)
    addfile_list = process.stdout.readlines()
    for ifile in range(len(addfile_list)):
        addfile_list[ifile] = addfile_list[ifile].strip()
        # Split to retain only the file name, remove path
        addfile_list[ifile] = addfile_list[ifile].split('/')[-1]
    # Combine the results
    file_list.extend(addfile_list)
    return file_list

file_list = get_file_list()

In [3]:
scdir = '/glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/'

test_str = ["CTL","HOMO_RAD"]
pclass_names = ['DC', 'CG', 'SC', 'ST', 'AN']
# pclass_names=["Deep", "Congestus", "Shallow", "Stratiform", "Anvil"]#, "DSA"] # DSA isn't there yet

In [10]:
PE_massFlux = {}

for expName in test_str:

    PE_massFlux[expName] = {}
    for iname in pclass_names:
        PE_massFlux[expName][iname] = []

    for ifile in file_list:

        parts = ifile.split('.')
        date = parts[1]
        pickle_file = f"{scdir}PE_massFlux_{expName}_{date}.pickle"

        # Read in variables from pickle
        try:
            with open(pickle_file, 'rb') as f:
                PE_thisExp = pickle.load(f)

            for ikey in PE_thisExp.keys():
                PE_massFlux[expName][ikey].append(PE_thisExp[ikey])
        except:
            print('File not ready: ',pickle_file)
            pass

    for ikey in PE_thisExp.keys():
        PE_massFlux[expName][ikey] = np.array(PE_massFlux[expName][ikey])

File not ready:  /glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/PE_massFlux_HOMO_RAD_2000-05-08_06.pickle
File not ready:  /glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/PE_massFlux_HOMO_RAD_2000-05-08_12.pickle
File not ready:  /glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/PE_massFlux_HOMO_RAD_2000-05-08_18.pickle
File not ready:  /glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/PE_massFlux_HOMO_RAD_2000-05-09_00.pickle
File not ready:  /glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/PE_massFlux_HOMO_RAD_2000-05-09_06.pickle
File not ready:  /glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/PE_massFlux_HOMO_RAD_2000-05-09_12.pickle
File not ready:  /glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/PE_massFlux_HOMO_RAD_2000-05-09_18.pickle
File not ready:  /glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/PE_massFlux_HOMO_RAD_2000-05-10_00.pickle
File not ready:  /glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/PE_massFlux_HOMO_RAD_2000-05-10_06.pickle
F

### Plotting

#### Time series

In [ ]:
# Function to compute running mean
def running_mean_conf(time_series):
    nd_smooth    = 6 # days
    nd_window_ci = 6
    # nd_smooth    = 20 # days
    # nd_window_ci = 20
    ntpday = 4 # timesteps per day
    window_size = nd_smooth*ntpday  # Adjust as needed
    window_size_ci = nd_window_ci*ntpday  # Adjust as needed
    tser_smooth = np.convolve(time_series, np.ones(window_size) / window_size, mode='valid')
    tser_ci = np.convolve(time_series, np.ones(window_size_ci) / window_size_ci, mode='valid')
    # Compute standard error of the mean
    standard_error = stats.sem(time_series)  # Standard error of the original data
    z_score = 1.96 # Z-score for 95% confidence interval
    confidence_interval = z_score * standard_error  # 95% confidence interval
    # confidence_interval = np.zeros_like(tser_ci)
    # # for i in range(len(tser_smooth)):
    # for i in range(len(tser_ci)):
    #     subset = time_series[i:i+window_size_ci]
    #     standard_error = stats.sem(subset)
    #     confidence_interval[i] = z_score * standard_error
    #     # Q1 = np.percentile(subset, 25)
    #     # Q3 = np.percentile(subset, 75)
    #     # IQR = Q3 - Q1
    #     # confidence_interval[i] = IQR/2
    return tser_smooth, confidence_interval, window_size, tser_ci, window_size_ci

In [ ]:
def plot_time_series(title, all_vars_imean, subtitles=None, units=None, ctlanom=False,
                     conf_shading=True, do_legend=True):
    nprof = all_vars_imean.shape[0]
    i_nt = all_vars_imean.shape[-1]
    # create figure
    fig_x = 7
    fig_y = 2.0*nprof + 1
    fig, axs = plt.subplots(nprof,1, figsize=(fig_x,fig_y), layout="constrained", dpi=300) # row, column
    # fig.suptitle(title)
    for iprof, ax in enumerate(axs):
        pltvar = np.copy(all_vars_imean[iprof])
        if ctlanom:
            pltvar -= pltvar[:,0,:][:,np.newaxis,:]
        # Plot all sensitivity tests for variable
        for itest in range(ntest):
            tser = pltvar[:,itest,:]
            # Smooth time series
            tser = running_mean_conf(tser)
            mean, low, high = mean_confidence_interval(tser)
            ax.plot(mean, linestyle=linestyle[itest], color=linecolor[itest], label=tests_str[itest])
            if conf_shading:
                # if (itest == 0) or (itest == 1):
                xdim = range(0,i_nt)
                ax.fill_between(xdim, high, low, alpha=0.2, color=linecolor[itest])
        ax.set_title(subtitles[iprof])
        ax.set_ylabel(units[iprof])
        if iprof < nprof-1:
            sns.despine(offset=10,ax=ax, bottom=True)
            # Remove tick labels
            ax.set_xticks([])
        else:
            sns.despine(offset=10,ax=ax)
            ax.set_xlabel('Time [hours]')
        if do_legend and iprof == 0:
            ax.legend(loc="lower left", frameon=False, fontsize=12)#, bbox_to_anchor=(0.05, 0.05))
    plt.show()